In [3]:
import os

def read_cases(file_path):
    with open(file_path, 'r') as f:
        cases = {line.strip() for line in f if line.strip()}
    return cases

def main():
    all_fold_sets = []
    fold_names = []

    for fold in range(1, 6):
        train_file = f"train_cases_{fold}.txt"
        test_file = f"test_cases_{fold}.txt"

        if not os.path.exists(train_file) or not os.path.exists(test_file):
            print(f"[!] Fichier manquant pour le fold {fold}")
            continue

        train_cases = read_cases(train_file)
        test_cases = read_cases(test_file)
        combined_cases = train_cases.union(test_cases)

        all_fold_sets.append(combined_cases)
        fold_names.append(f"Fold {fold}")

        print(f"[✓] {fold_names[-1]} : {len(combined_cases)} sujets")

    # Référence : fold 1
    reference = all_fold_sets[0]
    all_identical = True

    for i, current_set in enumerate(all_fold_sets[1:], start=2):
        diff_missing = reference - current_set
        diff_extra = current_set - reference

        if diff_missing or diff_extra:
            print(f"\n[✗] Différences dans Fold {i} par rapport à Fold 1 :")
            if diff_missing:
                print(f"   - Sujets manquants : {sorted(diff_missing)}")
            if diff_extra:
                print(f"   - Sujets en trop : {sorted(diff_extra)}")
            all_identical = False

    if all_identical:
        print("\n[✅] Tous les folds contiennent exactement les mêmes sujets.")

if __name__ == "__main__":
    main()


[✓] Fold 1 : 80 sujets
[✓] Fold 2 : 80 sujets
[✓] Fold 3 : 80 sujets
[✓] Fold 4 : 80 sujets
[✓] Fold 5 : 80 sujets

[✅] Tous les folds contiennent exactement les mêmes sujets.


In [8]:
import os
from collections import Counter, defaultdict

def read_cases_list(file_path):
    with open(file_path, 'r') as f:
        return [line.strip() for line in f if line.strip()]

# Charger les sujets sarcopéniques
sarcopenia_subjects = set(read_cases_list("sarcopenia_subj_DIASEM.txt"))

issues_found = False

triplet_groups = defaultdict(list)

# Suffixes de triplets à détecter
triplet_suffixes = ["_pad", "_faible", "_fort"]

def get_base_subject(name):
    for suffix in triplet_suffixes:
        if name.endswith(suffix):
            return name[:-len(suffix)]
    return None  # Pas un triplet

for fold in range(1, 6):
    train_file = f"train_cases_{fold}.txt"
    test_file = f"test_cases_{fold}.txt"

    if not os.path.exists(train_file) or not os.path.exists(test_file):
        print(f"[!] Fichier manquant pour le fold {fold}")
        continue

    train_cases = read_cases_list(train_file)
    test_cases = read_cases_list(test_file)

    # Vérif doublons dans train et test individuellement
    train_duplicates = [c for c, count in Counter(train_cases).items() if count > 1]
    test_duplicates = [c for c, count in Counter(test_cases).items() if count > 1]

    if train_duplicates:
        print(f"[✗] Doublons dans {train_file} : {sorted(train_duplicates)}")
        issues_found = True
    if test_duplicates:
        print(f"[✗] Doublons dans {test_file} : {sorted(test_duplicates)}")
        issues_found = True

    # Vérif qu'aucun sujet n'est dans train ET test du même fold
    intersection_fold = set(train_cases).intersection(set(test_cases))
    if intersection_fold:
        print(f"[✗] Sujets présents dans train ET test pour le fold {fold} : {sorted(intersection_fold)}")
        issues_found = True

    # Vérif sujets sarcopéniques dans train et test
    sarc_in_train = sarcopenia_subjects.intersection(train_cases)
    sarc_in_test = sarcopenia_subjects.intersection(test_cases)

    if sarc_in_train:
        print(f"[✗] Sujets sarcopéniques dans {train_file} : {sorted(sarc_in_train)}")
        issues_found = True
    if sarc_in_test:
        print(f"[✗] Sujets sarcopéniques dans {test_file} : {sorted(sarc_in_test)}")
        issues_found = True

    # Enregistrer les triplets pour vérification
    for case in train_cases:
        base = get_base_subject(case)
        if base is not None:
            triplet_groups[(fold, base)].append(("train", case))
    for case in test_cases:
        base = get_base_subject(case)
        if base is not None:
            triplet_groups[(fold, base)].append(("test", case))

# Vérif triplets regroupés dans même split
for (fold, base), cases in triplet_groups.items():
    splits = {split for split, _ in cases}
    if len(splits) > 1:
        print(f"[✗] Triplet {base} réparti entre train et test dans fold {fold}: {cases}")
        issues_found = True

if not issues_found:
    print("[✅] Toutes les vérifications sont OK :")
    print(" - Pas de doublons dans chaque fichier")
    print(" - Aucun sujet n’apparaît dans train et test du même fold")
    print(" - Aucun sujet sarcopénique dans les fichiers")
    print(" - Les triplets (_pad, _faible, _fort) sont bien regroupés dans train ou test")


[✅] Toutes les vérifications sont OK :
 - Pas de doublons dans chaque fichier
 - Aucun sujet n’apparaît dans train et test du même fold
 - Aucun sujet sarcopénique dans les fichiers
 - Les triplets (_pad, _faible, _fort) sont bien regroupés dans train ou test


In [9]:
triplet_groups

defaultdict(list,
            {(1, 'sujet10_RF'): [('train', 'sujet10_RF_faible'),
              ('train', 'sujet10_RF_fort'),
              ('train', 'sujet10_RF_pad')],
             (1, 'sujet11_RF'): [('train', 'sujet11_RF_faible'),
              ('train', 'sujet11_RF_fort'),
              ('train', 'sujet11_RF_pad')],
             (1, 'sujet13_RF'): [('train', 'sujet13_RF_faible'),
              ('train', 'sujet13_RF_fort'),
              ('train', 'sujet13_RF_pad')],
             (1, 'sujet14_RF'): [('train', 'sujet14_RF_faible'),
              ('train', 'sujet14_RF_fort'),
              ('train', 'sujet14_RF_pad')],
             (1, 'sujet15_RF'): [('train', 'sujet15_RF_faible'),
              ('train', 'sujet15_RF_fort'),
              ('train', 'sujet15_RF_pad')],
             (1, 'sujet1_RF'): [('train', 'sujet1_RF_faible'),
              ('train', 'sujet1_RF_fort'),
              ('train', 'sujet1_RF_pad')],
             (1, 'sujet2_RF'): [('train', 'sujet2_RF_faible'),
     